In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import h5py
import pandas as pd
import os

# Open the HDF5 file
with h5py.File('/content/drive/MyDrive/gist-960-euclidean.hdf5', 'r') as f:
    # List all keys (datasets) in the file
    print("Keys in the HDF5 file:", list(f.keys()))

    # Read the training set, queries, ground truth, and distances
    train_data = f['train'][:]  # Assuming 'train' is the key for training data
    query_data = f['test'][:]  # Assuming 'test' is the key for query data
    ground_truth = f['neighbors'][:]  # Assuming 'neighbors' is the key for ground truth neighbors
    distance = f['distances'][:]  # Assuming 'distances' is the key for ground truth distances

# Inspect the data
print("Training data shape:", train_data.shape)
print("Query data shape:", query_data.shape)
print("Ground truth shape:", ground_truth.shape)
print("Distance truth shape:", distance.shape)


Keys in the HDF5 file: ['distances', 'neighbors', 'test', 'train']
Training data shape: (1000000, 960)
Query data shape: (1000, 960)
Ground truth shape: (1000, 100)
Distance truth shape: (1000, 100)


In [10]:
import numpy as np
from typing import List, Tuple, Optional, Any
from collections import defaultdict
import heapq
from concurrent.futures import ThreadPoolExecutor
import threading

class MTreeNode:
    def __init__(self, data=None, radius=0, parent=None, distance_to_parent=0):
        self.data = data
        self.radius = radius
        self.parent = parent
        self.distance_to_parent = distance_to_parent
        self.routing_entries = []
        self.is_leaf = True
        self._distance_cache = {}
        self._lock = threading.Lock()

    def add_routing_entry(self, child, distance):
        self.routing_entries.append((child, distance))
        self.is_leaf = False

class MTree:
    def __init__(self, max_node_size=4, distance_metric='euclidean'):
        self.root = None
        self.max_node_size = max_node_size
        self.distance_metric = distance_metric
        self._distance_cache = {}
        self._cache_lock = threading.Lock()

    def _compute_distance(self, x1: np.ndarray, x2: np.ndarray) -> float:
        key = tuple(sorted([hash(x1.tobytes()), hash(x2.tobytes())]))

        with self._cache_lock:
            if key in self._distance_cache:
                return self._distance_cache[key]

            if self.distance_metric == 'euclidean':
                dist = np.sqrt(np.sum((x1 - x2) ** 2))
            else:
                dist = np.linalg.norm(x1 - x2)

            self._distance_cache[key] = dist
            return dist

    def bulk_load(self, data: np.ndarray, batch_size: int = 10000):
        print("Starting bulk load...")
        n_samples = len(data)
        n_batches = (n_samples + batch_size - 1) // batch_size

        def process_batch(batch_data):
            return MTreeNode(data=batch_data)

        nodes = []
        with ThreadPoolExecutor() as executor:
            futures = []
            for i in range(0, n_samples, batch_size):
                batch = data[i:i + batch_size]
                futures.append(executor.submit(process_batch, batch))

            for future in futures:
                nodes.append(future.result())

        while len(nodes) > 1:
            new_nodes = []
            for i in range(0, len(nodes), self.max_node_size):
                batch = nodes[i:i + self.max_node_size]
                if len(batch) == 1:
                    new_nodes.append(batch[0])
                    continue

                parent = MTreeNode()
                pivot = batch[0]
                parent.data = pivot.data

                def compute_distances(node):
                    return self._compute_distance(parent.data, node.data)

                with ThreadPoolExecutor() as executor:
                    distances = list(executor.map(compute_distances, batch))

                for node, distance in zip(batch, distances):
                    node.parent = parent
                    node.distance_to_parent = distance
                    parent.add_routing_entry(node, distance)

                parent.radius = max(distances)
                new_nodes.append(parent)

            nodes = new_nodes
            print(f"Created {len(nodes)} nodes at this level")

        self.root = nodes[0]
        print("Bulk loading completed")

    def _search_knn(self, query: np.ndarray, k: int) -> List[Tuple[float, np.ndarray]]:
        if not self.root:
            return []

        pq = []
        candidates = []

        def process_node(node, query_dist, current_radius): # Pass current_radius as argument
            # Initialize radius here with current_radius
            radius = current_radius

            if node.is_leaf:
                dist = self._compute_distance(query, node.data)
                candidates.append((dist, node.data))
                if len(pq) < k:
                    heapq.heappush(pq, (-dist, node.data))
                    radius = -pq[0][0] if len(pq) == k else float('inf') # Update radius
                elif dist < radius:
                    heapq.heapreplace(pq, (-dist, node.data))
                    radius = -pq[0][0] # Update radius
                return radius # Return updated radius
            else:
                entries = []
                for child, child_dist in node.routing_entries:
                    min_dist = max(0, query_dist - child.radius)
                    entries.append((min_dist, child, child_dist))

                entries.sort(key=lambda x: x[0])

                for min_dist, child, child_dist in entries:
                    if min_dist <= radius:
                        child_query_dist = self._compute_distance(query, child.data)
                        if child_query_dist - child.radius <= radius:
                            radius = process_node(child, child_query_dist, radius) # Pass and return radius
                return radius # Return updated radius

        initial_radius = float('inf')
        root_dist = self._compute_distance(query, self.root.data)
        final_radius = process_node(self.root, root_dist, initial_radius) # Pass initial_radius

        # Sort candidates by distance and return k nearest
        candidates.sort(key=lambda x: x[0])
        return candidates[:k]

    def knn_search(self, queries: np.ndarray, k: int) -> List[List[Tuple[float, np.ndarray]]]:
        results = []
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(self._search_knn, query, k) for query in queries]
            results = [future.result() for future in futures]
        return results

In [12]:
mtree = MTree(max_node_size=20)
mtree.bulk_load(train_data)

Starting bulk load...
Created 5 nodes at this level
Created 1 nodes at this level
Bulk loading completed


In [20]:
results = mtree.knn_search(query_data[:15], k=100)

In [21]:
import numpy as np
from collections import defaultdict

def create_vector_to_index_mapping(train_data, decimals=10):
    """
    Create a mapping from vector tuple to index for fast lookup.

    Args:
        train_data: numpy array of shape (1000000, 960)
        decimals: number of decimal places to round to
    """
    # Round the data to handle floating point precision
    rounded_data = np.round(train_data, decimals=decimals)

    # Create mapping of vector to index
    vector_to_idx = {}
    for idx, vector in enumerate(rounded_data):
        vector_to_idx[tuple(vector)] = idx

    return vector_to_idx

def batch_find_indices(vectors, vector_to_idx_map, decimals=10):
    """
    Find indices for multiple vectors at once using the precomputed mapping.

    Args:
        vectors: numpy array of vectors
        vector_to_idx_map: dict mapping vector tuples to indices
        decimals: number of decimal places to round to
    """
    rounded_vectors = np.round(vectors, decimals=decimals)
    indices = []
    for v in rounded_vectors:
        indices.append(vector_to_idx_map.get(tuple(v), -1))
    return np.array(indices)

def evaluate_knn_accuracy_optimized(knn_results, ground_truth_indices, train_data):
    """
    Optimized version of KNN accuracy evaluation.

    Args:
        knn_results: numpy array of shape (num_queries, num_neighbors) containing neighbor vectors
        ground_truth_indices: numpy array of shape (num_queries, num_neighbors) containing true indices
        train_data: numpy array of shape (num_train, vector_dim) containing training data
    """
    # Create vector to index mapping once
    vector_to_idx_map = create_vector_to_index_mapping(train_data)

    num_queries = len(knn_results)
    num_neighbors = len(knn_results[0])

    # Process each query point
    predicted_indices = []
    for query_neighbors in knn_results:
        # Find indices for all neighbors of this query point
        query_indices = batch_find_indices(query_neighbors, vector_to_idx_map)
        predicted_indices.append(query_indices)

    predicted_indices = np.array(predicted_indices)

    # Compute accuracy using vectorized operations
    correct_predictions = np.sum([
        np.isin(ground_truth_indices[i], predicted_indices[i]).sum()
        for i in range(num_queries)
    ])

    accuracy = correct_predictions / (num_queries * num_neighbors)
    return accuracy



In [2]:
accuracy = evaluate_knn_accuracy_optimized(results, ground_truth[:15], train_data)
print("Accuracy : ",accuracy)

Accuracy :  97.52
